# Hands On Example of MLFlow on a ML lifecycle

In [1]:
import warnings

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3934,7.3,0.27,0.30,1.3,0.040,26.0,84.0,0.99222,3.28,0.53,10.7,6
4228,6.2,0.29,0.23,12.4,0.048,33.0,201.0,0.99612,3.11,0.56,9.9,6
2130,7.0,0.35,0.24,1.9,0.040,21.0,144.0,0.99230,3.35,0.38,11.0,5
1183,6.4,0.26,0.30,2.2,0.025,33.0,134.0,0.99200,3.21,0.47,10.6,6
2534,6.8,0.23,0.30,1.7,0.043,19.0,95.0,0.99207,3.17,0.46,10.7,7
462,5.9,0.25,0.19,12.4,0.047,50.0,162.0,0.99730,3.35,0.38,9.5,5
2949,6.0,0.28,0.52,5.0,0.078,30.0,139.0,0.99494,3.10,0.36,9.0,6
2228,7.1,0.20,0.36,11.6,0.042,45.0,124.0,0.99700,2.92,0.59,9.5,7
3158,7.1,0.36,0.37,4.8,0.019,39.0,114.0,0.99036,3.08,0.49,12.7,7
777,7.2,0.23,0.19,13.7,0.052,47.0,197.0,0.99865,3.12,0.53,9.0,5


## Launching mlflow server

Command to launch mlflow server:

`mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000`

In [3]:
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [4]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [5]:
# Creating new experiment to store
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2023/12/19 15:39:00 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/aujasvi/TAMLEP/handson-mlflow/mlruns/326144508528889643', creation_time=1702980540995, experiment_id='326144508528889643', last_update_time=1702980540995, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

## ML Runs

In [6]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, "model")

### Function calls

In [7]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: /home/aujasvi/TAMLEP/handson-mlflow/mlruns/326144508528889643/1e5494db20f04d4d9b84cdc28d51c95c/artifacts


In [8]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: /home/aujasvi/TAMLEP/handson-mlflow/mlruns/326144508528889643/19937452e16b4daca746df75f0f3a623/artifacts


In [9]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/aujasvi/TAMLEP/handson-mlflow/mlruns/326144508528889643/d589a224470348a7893bf8ee6fb799d7/artifacts


To run the MLFlow project on Default experiment:

`mlflow run . -P alpha=0.42`